# Trabalho prático - Predição de Spam


## 1) Bibliotecas e funções usadas

### 1.1) Bibliotecas necessárias 

In [25]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn import svm


color = sns.color_palette()

### 1.2) Funções úteis

In [3]:
# Gráficos de barras 
def barChart(x, y, title):
    # This will plot a simple bar chart
    plt.bar(x, y)
 
    # Title to the plot
    plt.title(title)
 
    # Adding the legends
    plt.legend(["bar"])
    plt.show()

# Gráfico de pontos 
def ScatterChart(x, y, title):
    fig = px.scatter(spam, x, y)
    fig.update_traces(marker_color="turquoise",marker_line_color='rgb(8,48,107)',
                    marker_line_width=1.5)
    fig.update_layout(title_text=title)
    fig.show()

## 2) Carregar dados do DB

In [15]:
nomes = ['word_freq_make', 'word_freq_address', 'word_freq_all', 'word_freq_3d', 'word_freq_our', 'word_freq_over', 'word_freq_remove', 'word_freq_internet', 'word_freq_order', 'word_freq_mail', 'word_freq_receive', 'word_freq_will', 'word_freq_people', 'word_freq_report', 'word_freq_addresses', 'word_freq_free', 'word_freq_business', 'word_freq_email', 'word_freq_you', 'word_freq_credit', 'word_freq_your',	'word_freq_font', 'word_freq_000', 'word_freq_money', 'word_freq_hp', 'word_freq_hpl',	'word_freq_george',	'word_freq_650', 'word_freq_lab', 'word_freq_labs', 'word_freq_telnet', 'word_freq_857', 'word_freq_data', 'word_freq_415', 'word_freq_85', 'word_freq_technology',	'word_freq_1999', 'word_freq_parts', 'word_freq_pm', 'word_freq_direct', 'word_freq_cs', 'word_freq_meeting', 'word_freq_original', 'word_freq_project', 'word_freq_re', 'word_freq_edu', 'word_freq_table', 'word_freq_conference', 'char_freq_%3B', 'char_freq_%28', 'char_freq_%5B', 'char_freq_%21', 'char_freq_%24', 'char_freq_%23', 'capital_run_length_average', 'capital_run_length_longest', 'capital_run_length_total', 'class']

spam = pd.read_csv("./spambase.data", names=nomes)

# Separar as classes dos atributos
X = spam[spam.columns.difference(['class'])].values # Matriz com as features 
y = spam['class'].values    # Vetor com as classes 

# print(spam.head())
# print(X)
# print(y)


## 3) Análise do DB


### 3.1) Visualização das classes do DB:
A visualização das classes permite observar que o Database é levemente não balanceado 

In [5]:
dist = spam['class'].value_counts()
colors = ['mediumturquoise', 'darkorange']
trace = go.Pie(values=(np.array(dist)),labels=dist.index)
layout = go.Layout(title='Spam')
data = [trace]
fig = go.Figure(trace,layout)
fig.update_traces(marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()

### 3.2) Normalização do DB
Através do método describe observa-se que os atributos não possuem escalas iguais. Logo, é necessários normalizá-los.

In [16]:
# print(spam.describe())
normPadrao = StandardScaler()

# Ajusta os dados e transforma eles
X_norm = normPadrao.fit_transform(X)


## 4) Modelos 

### 4.1) SVM
#### a) Hiperparâmetros

- Parêmetro C: Penalidade para cada ponto de dado classificado errado. Um C pequeno representa pouca penalidade para erros na classificação, enquanto que um valor alto desse parâmetro pode resultar em overfitting.
- Parâmetro Gamma: Para kernel não lineares o gamma define o grau de influência do kernel trick. Por exemplo, para a função kernel RBF o gamma representa a distância de influência de um único ponto de dados.

#### b) Métrica de avaliação: 
Visando o desbanlanceamento da classes a métrica utilizada para avaliação será a precisão 

### 4.2) MLP

#### a) Hiperparâmetros 
- Número de camadas escondidas 
- Função de ativação 
- Taxa de aprendizado 
- Número de neurônio por camada 

#### b) Métrica de avaliação: 
Visando o desbanlanceamento da classes a métrica utilizada para avaliação será a precisão 


### 4.3) KNN
#### a) Hiperparâmetros 
- Número de vizinhos 
- Pesos (weights): Influência dos vizinhos no ponto de dados referente a classificação desse.
- Métrica: Forma de calcular a distência entre os pontos relativa a similiridade entre eles 

#### b) Métrica de avaliação: 
Visando o desbanlanceamento da classes a métrica utilizada para avaliação será a precisão 

### 5) SVM


#### 5.1) Treinar modelo com a validação cruzada aninhada


In [27]:
loops = 10
tunnig_par = {"C":[1, 5, 10, 15, 20, 25, 30, 35, 40], "gamma": [0.001, 0.01, 0.1, 1]}
accuracy_loops = np.zeros(loops)

svm_clf = svm.SVC(kernel='rbf')

# Algoritmo de validação cruzada aninhada com otimização de parâmetros
for i in range(loops):

    # Estrategia de validação cruzada - 10n10fold
    cv_estrategy = KFold(n_splits=10, shuffle=True, random_state=i)

    # otimização de parametros 
    svm_clf = GridSearchCV(estimator=svm_clf, param_grid=tunnig_par, cv=cv_estrategy)
    accuracy = cross_val_score(svm_clf, X=X_norm, y=y, cv=cv_estrategy)
    accuracy_loops[i] = accuracy.mean()

barChart(range(loops), accuracy_loops, 'Precisão ao longo dos n')

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/pedro/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/pedro/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py", line 874, in fit
    self._run_search(evaluate_candidates)
  File "/home/pedro/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py", line 1388, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "/home/pedro/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py", line 821, in evaluate_candidates
    out = parallel(

## Referências 
1) https://towardsdatascience.com/a-beginners-guide-to-data-analysis-in-python-188706df5447
2) https://medium.com/@tonop15/spambase-data-exploration-analysis-9a3d6d83ee78
3) https://towardsdatascience.com/hyperparameter-tuning-for-support-vector-machines-c-and-gamma-parameters-6a5097416167
4) https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/
5) https://scikit-learn.org/stable/index.html
6) https://www.analyticsvidhya.com/blog/2021/05/tuning-the-hyperparameters-and-layers-of-neural-network-deep-learning/
7) https://panjeh.medium.com/scikit-learn-hyperparameter-optimization-for-mlpclassifier-4d670413042b
8) https://www.kaggle.com/code/arunimsamudra/k-nn-with-hyperparameter-tuning
9) https://www.v7labs.com/blog/train-validation-test-set
10) https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html#sphx-glr-auto-examples-model-selection-plot-nested-cross-validation-iris-py
